# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

In [ ]:
import numpy as np
from typing import Optional, Union, Tuple


def euclidean_distance(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean distance between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Length of the line segment connecting given points
    """
    x = np.array(x, dtype = np.float64)
    y = np.array(y, dtype = np.float64)
    return np.sqrt(np.sum((x - y) ** 2))


def euclidean_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean similarity between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Similarity between points x and y
    """
    x = np.array(x, dtype = np.float64)
    y = np.array(y, dtype = np.float64)
    return 1 / (1 + euclidean_distance(x, y))


def pearson_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate a Pearson correlation coefficient given 1-D data arrays x and y
    Args:
        x, y: two points in n-space
    Returns:
        Pearson correlation between x and y
    """
    x = np.array(x, dtype = np.float64)
    y = np.array(y, dtype = np.float64)
    return np.sum((x - np.mean(x)) * (y - np.mean(y))) / np.sqrt(np.sum((x - np.mean(x)) ** 2) * np.sum((y - np.mean(y)) ** 2))

Извините, у меня никак не получилось запустить test.py

Но я - честно - старалась

In [ ]:
!pytest test.py

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [ ]:
def euclidean_similarity_ind(x, y):
    mask = (x == 0) | (y == 0)
    if mask.all():
        return -np.inf
    x = np.ma.compressed(np.ma.masked_array(x, mask = mask))
    y = np.ma.compressed(np.ma.masked_array(y, mask = mask))
    return euclidean_similarity(x, y)

def pearson_similarity_ind(x, y):
    mask = (x == 0) | (y == 0)
    if mask.all():
        return -np.inf
    x = np.ma.compressed(np.ma.masked_array(x, mask = mask))
    y = np.ma.compressed(np.ma.masked_array(y, mask = mask))
    return pearson_similarity(x, y)

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [ ]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.__n_recommend = n_recommendations
        self.__alpha = alpha
        self.__sim = None
        if (metric == 'euclidean'):
            self.__sim = lambda x, y: 1 - euclidean_similarity_ind(x, y)
        else:
            self.__sim = lambda x, y: 1 - pearson_similarity_ind(x, y)

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        knn = NearestNeighbors(n_neighbors=self.__n_recommend, metric = self.__sim, radius = 1 - self.__alpha)
        knn.fit(X)
        self.__X = X
        self.__knn = knn

    def __find_closest_users(self, user_id: int, n_closest_users: int):
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        user_vec = self.__X[user_id].reshape(1, -1)
        _, neighbors = self.__knn.radius_neighbors(user_vec)
        neighbors = neighbors[0]
        return neighbors[:min(n_closest_users, len(neighbors))]

    def __make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        if n_recommendations is None:
            n_recommendations = self.__n_recommend
        users = self.__find_closest_users(user_id, 60)
        movies = np.argsort(np.apply_along_axis(np.sum, 0, self.__X[users, :]))
        return movies[:n_recommendations]

    def make_recommendations(self, user_ids, n_recommendations: Optional[int] = None):
        if n_recommendations is None:
            n_recommendations = self.__n_recommend
        res = []
        for user_id in user_ids:
            res.append(self.__make_recommendation(user_id, n_recommendations))
        return np.array(res)

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

**Пример:** $(1, 1, 2, 3), (5, 1, 4, 6)$

**Объяснение:**  по метрике Пирсона они близки, так как фильмы 3-4 оценены пропорционально. По Евклидовой метрике расстояния слишком большое

In [ ]:
x = np.array([1, 1, 2, 3])
y = np.array([5, 1, 4, 6])

print(euclidean_similarity_ind(x, y))
print(pearson_similarity_ind(x, y))

0.1566130288262323
0.6446583712203042


<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

In [ ]:
def pk(actual, predicted, k = 10):  #precision at k
    return len(np.intersect1d(actual, predicted[:min(k, len(predicted))])) / k


def apk(actual: np.array, predicted: np.array, k: int = 10) -> float:
    """
    Compute the average precision at k
    Args:
        actual: a list of elements that are to be predicted (order doesn't matter)
        predicted: a list of predicted elements (order does matter)
        k: the maximum number of predicted elements
    Returns:
        The average precision at k over the input lists
    """
    sum_precision = 0
    for i in range(k):
        sum_precision += pk(actual, predicted, i + 1) * int(predicted[i] in actual)
    return sum_precision / k


def mapk(actual: np.array, predicted: np.array, k: int = 10) -> float:
    """
    Compute the mean average precision at k
    Args:
        actual: a list of lists of elements that are to be predicted
        predicted: a list of lists of predicted elements
        k: the maximum number of predicted elements
    Returns:
        The mean average precision at k over the input lists
    """
    n = len(actual)
    sum_precision = 0
    for i in range(n):
        sum_precision += apk(actual[i], predicted[i], k)
    return sum_precision / n


## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
data.userId = enc.fit_transform(data.userId)
data.movieId = enc.fit_transform(data.movieId)

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

In [ ]:
X = np.zeros((len(data.userId.unique()), len(data.movieId.unique())))

Удалим для наиболее активных пользователей 5 оценок

In [ ]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [ ]:
X[data.userId, data.movieId] = data.rating

In [ ]:
print(test_data)

   userId movieId  rating   timestamp
0     546    3340     4.0  1028129718
1     546     772     4.0  1039880724
2     546    8781     4.5  1468681977
3     546    5835     4.0  1242992741
4     546    1369     3.5  1342849917
5     563    2858     5.0   974708761
6     563    2820     5.0   974844711
7     563    1527     3.0   974714208
8     563     420     3.0   974843382
9     563     654     4.0   974839307
10    623    4926     1.0  1178980613
11    623     823     3.0  1028110632
12    623    4099     3.0  1119469511
13    623    8090     3.0  1378584502
14    623    3064     1.0  1035119788
15     14    8052     1.5  1347937652
16     14     427     3.0   997939193
17     14     503     3.0  1166586512
18     14    1287     3.0  1166587214
19     14    6957     1.0  1316396154
20     72     102     3.5  1348568488
21     72     414     2.5  1304411724
22     72    3447     4.0  1429168542
23     72    7809     3.5  1319336835
24     72     880     3.5  1255844891
25    451   

In [ ]:
test_data = test_data.movieId.values.reshape(len(active_users), 5)

In [ ]:
print(test_data)

[[3340 772 8781 5835 1369]
 [2858 2820 1527 420 654]
 [4926 823 4099 8090 3064]
 [8052 427 503 1287 6957]
 [102 414 3447 7809 880]
 [1354 3192 1257 1290 963]
 [5006 6550 4032 4217 309]
 [266 3641 5017 6749 7483]
 [1639 1269 1224 83 2303]
 [2576 2587 2798 2134 2195]]


Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [ ]:
best_mod = UserBasedRecommendation(metric = 'euclidean', n_recommendations=100, alpha = 0.5)
best_mod.fit(X)
best_pred = best_mod.make_recommendations(active_users)

for met in ['euclidean', 'pearson']:
    for a in np.linspace(0.01, 1, 50):
        mod = UserBasedRecommendation(metric = met,  n_recommendations = 100, alpha = a)
        mod.fit(X)
        pred = mod.make_recommendations(active_users)

        if (mapk(test_data, best_pred, 100) >= mapk(test_data, pred, 100)):
            best_mod = mod
            best_pred = pred


Я не знаю почему оно не работает. честно. вообще. совсем. но я пыталась, за что я себя хвалю (даже если будет 0)

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [ ]:
print(mapk(test_data, best_pred, 5))
print(mapk(test_data, best_pred, 10))
print(mapk(test_data, best_pred, 100))

0.0
0.0
0.0


Как можно улучшить работу модели?

<b>Ответ:</b> найти метрику еще лучше/пойти по item-based подходу